In [1]:
import yfinance
import ta
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
# ML
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.models.experimental import SharpnessAwareMinimization
from tensorflow.keras.utils import plot_model
import gc

In [21]:
amzn = yfinance.download('AMZN', period='10y', interval='1d', prepost=True, threads=True).drop('Adj Close', axis=1)
amzn = ta.add_all_ta_features(amzn, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

[*********************100%***********************]  1 of 1 completed


C:\Program Files\Python39\lib\site-packages\ta\trend.py:730: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  self._trs[0] = diff_directional_movement.dropna()[
C:\Program Files\Python39\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Program Files\Python39\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [82]:
def windowed_dataset(series, window_size, batch_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(len(series))
    ds = ds.map(lambda w: (w, int(w[-1][3]/w[0][3] - 1 <= -0.1)))  # 1 is drop more than 10%, 0 is not
    return ds.batch(batch_size).prefetch(-1)

window_size = 30
batch_size = 64
amzn_ds = windowed_dataset(amzn, window_size, batch_size)
# for x, y in amzn_ds.take(1):
#     print(x.shape, y.shape)

In [83]:
tf.keras.backend.clear_session()
xIn = Input(shape=(window_size, len(amzn.columns)))
x = Bidirectional(LSTM(32, return_sequences=True))(xIn)
x = LayerNormalization()(x)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x = LayerNormalization()(x)
x = GlobalAvgPool1D()(x)
x_d = GlobalAvgPool1D()(xIn)
x_d = Dense(64, activation='swish')(x_d)
x_d = LayerNormalization()(x_d)
x_d = Dense(64, activation='swish')(x_d)
x_d = LayerNormalization()(x_d)
x_d = Dropout(0.2)(x_d)
x_c = Concatenate()([x, x_d])
x_c = Dense(64, activation='swish')(x_c)
x_c = LayerNormalization()(x_c)
xOut = Dense(1, activation='sigmoid')(x_c)

model = Model(xIn, xOut)
model.summary()
plot_model(model, show_shapes=True, to_file='risk_model.png')
model = SharpnessAwareMinimization(model)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 91)]     0           []                               
                                                                                                  
 global_average_pooling1d_1 (Gl  (None, 91)          0           ['input_1[0][0]']                
 obalAveragePooling1D)                                                                            
                                                                                                  
 bidirectional (Bidirectional)  (None, 30, 64)       31744       ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 64)           5888        ['global_average_pooling1d_1[

In [84]:
epoch = 300
callbacks = [
    LearningRateScheduler(CosineDecay(1e-3, epoch*batch_size)),
    EarlyStopping(monitor='loss', patience=40, restore_best_weights=True),
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=20, min_lr=1e-7)
]

hist = model.fit(amzn_ds,  batch_size=batch_size, epochs=100, verbose=1, callbacks=callbacks, use_multiprocessing=True)
print(hist.epoch[-1], max(hist.history['accuracy']))  # show final epoch no and lowest rmse

Epoch 1/100
39/39 [==============================] - 11s 28ms/step - loss: 0.4035 - accuracy: 0.8834 - lr: 0.0010
Epoch 2/100
39/39 [==============================] - 3s 66ms/step - loss: 0.2929 - accuracy: 0.9055 - lr: 0.0010
Epoch 3/100
39/39 [==============================] - 1s 18ms/step - loss: 0.2743 - accuracy: 0.9055 - lr: 0.0010
Epoch 4/100
39/39 [==============================] - 1s 17ms/step - loss: 0.2657 - accuracy: 0.9067 - lr: 1.0000e-03
Epoch 5/100
39/39 [==============================] - 1s 18ms/step - loss: 0.2672 - accuracy: 0.9071 - lr: 1.0000e-03
Epoch 6/100
39/39 [==============================] - 1s 18ms/step - loss: 0.2578 - accuracy: 0.9119 - lr: 1.0000e-03
Epoch 7/100
39/39 [==============================] - 1s 17ms/step - loss: 0.2521 - accuracy: 0.9140 - lr: 1.0000e-03
Epoch 8/100
39/39 [==============================] - 1s 18ms/step - loss: 0.2478 - accuracy: 0.9144 - lr: 1.0000e-03
Epoch 9/100
39/39 [==============================] - 1s 19ms/step - loss: 0

In [85]:
model.save('amzn risk model 96.58')

INFO:tensorflow:Assets written to: amzn risk model 96.58\assets


INFO:tensorflow:Assets written to: amzn risk model 96.58\assets
